In [51]:
import pandas as pd
import numpy as np
import Azim_Utility

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
import sklearn.tree
from sklearn.datasets import make_classification
from sklearn.model_selection import GridSearchCV

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.metrics import classification_report,confusion_matrix, accuracy_score

##Seaborn for fancy plots. 
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams["figure.figsize"] = (4,4)

## 3950 Assignment 1: Part 2

For this assignment we want to use some sort of tree based model to classify the data below. We have a very small training set, so overfitting is a very real concern. 

Some specifics for this assignment:
<ul>
<li>Please use the show_eda to control if EDA stuff is shown. I don't really need to see all the EDA stuff (nor do you after you've done it), so we can make it configurable with a variable to speed up time. Please set this FALSE when you submit, so I can run all and see the outcome without histograms etc...
<li>Please ensure that whatever model you end up with is in a variable named best at the end.
<li>Please use some pipeline in prepping the data. The test data is in an identical format to the training data, so whatever pipeline you've created for your training will work for the testing. 
<li>The accuracy scoring will be an average of accuracy and roc_auc. 
</ul>

### Grading Metrics
<ul>
<li><b>Pipeline Used - 10pts</b> The data loading needs to be in a pipeline. See the test part for illustration. When testing I'll call your pipe with the new data (format is identical to training), so any prep stuff should be in the pipeline. 
<li><b>Tree Based Model Used - 5pts</b> The model used for classification needs to be some variety of tree, beyond that it is up to you. 
<li><b>Accuracy - 5pts</b> The final accuracy acheived. This will be a rough ranking, I'm assuming most people will get a similar level of accuracy, marks will only be deducted if yours is far wosrse, as that's an indication that you probably didn't take any/many steps to improve things. 
<li><b>Clarity and Formatting - 5pts</b> Is it organized and can I read it?
    <ul>
    <li> <b>Note:</b> for this assignment, and in general, please get rid of my comments and replace them with your own. I'm going to read this, so all of these instructions aren't really required. Think of this as a template, get rid of the stuff that isn't needed, and leave only the things you need to explain your code. 
    </ul>
</ul>

For submission, please drop the URL for your repository in the dropbox.

In [52]:
#Please change to your name.
name = "Azim's"

#Please use this to control EDA. 
show_eda = False

In [53]:
#Load data
df = pd.read_csv("training.csv")
df = df.drop(columns={"id"})
df.sample(5)

,target,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,...,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199,var_200
246,1,0.323,0.373,0.862,0.413,0.891,0.500,0.231,0.307,0.461,...,0.656,0.846,0.904,0.281,0.182,0.326,0.541,0.393,0.033,0.458
147,0,0.858,0.813,0.899,0.729,0.164,0.519,0.166,0.030,0.210,...,0.007,0.117,0.617,0.066,0.558,0.498,0.621,0.864,0.986,0.003
107,0,0.915,0.671,0.243,0.943,0.786,0.867,0.475,0.034,0.397,...,0.808,0.273,0.471,0.531,0.083,0.169,0.869,0.335,0.929,0.106
109,1,0.204,0.559,0.314,0.523,0.662,0.385,0.486,0.311,0.636,...,0.693,0.896,0.768,0.143,0.731,0.769,0.437,0.526,0.669,0.946
33,0,0.825,0.449,0.389,0.067,0.728,0.096,0.628,0.060,0.870,...,0.802,0.931,0.580,0.960,0.524,0.658,0.967,0.344,0.990,0.656


### Starting

For this assignment, you have a small training set, so combatting overfitting is key in being accurate!

In [46]:
df.shape

(250, 201)

In [54]:
test=df.copy()
# Creating my own test dataframe which will be my example of testing data the model has not seen.
np.random.seed(0)
test.drop(test.sample(frac=0.25).index, inplace=True) #randomly removes 25% of the rows
test.to_csv('azimTEST.csv', index=False) #save new file as azimTEST.csv
test.shape

(188, 201)

In [55]:
# Can not use our Azim_Utility.EDA() class because there are over 200 columns. So, .......

# Create function to quickly check the details on the datarfame. Detailes include:
# Number of Categorical Columns
# Number ofNumerical Columns
# Number of missing values
# Number of values < 0 to decide on what scaling method to use.
def chek_cols(data):
    Categorical_Columns=df.select_dtypes(include=['object']).columns
    Numerical_Columns=df.select_dtypes(include=['float64', 'int64']).columns
    if len(Categorical_Columns) > 0 and len(Numerical_Columns) == 0:
        print('Dataframe consists of only Categorical Columns.')
        print('Number of Catagorical Columns:',len(Categorical_Columns))
        print('Total NULL Values in Dataframe: ',(df.isnull().sum()).sum()) # checking and prining sum of null values 
    elif len(Categorical_Columns) == 0 and len(Numerical_Columns) > 0:
        print('Dataframe consists of only Numerical Columns.')
        print('Number of Numerical Columns:', len(Numerical_Columns))
        print('Total NULL Values in Dataframe: ',(df.isnull().sum()).sum()) # checking and prining sum of null values 
        print('Number of Numerical values in dataframe that are < 0 (negative values) :', ((df< 0).sum()).sum())
    else:
        print('Dataframe has a combination of Categorical and Numerical Columns.')
        print('Number of Catagorical Columns:',len(Categorical_Columns))
        print('Number of Numerical Columns:', len(Numerical_Columns))
        print('Total NULL Values in Dataframe: ',(df.isnull().sum()).sum()) # checking and prining sum of null values 
        print('Number of Numerical values < 0 (negative values):', ((df< 0).sum()).sum())
    return 
    
chek_cols(df)

Dataframe consists of only Numerical Columns.
Number of Numerical Columns: 201
Total NULL Values in Dataframe:  0
Number of Numerical values in dataframe that are < 0 (negative values) : 0


Which Tree Classification to use?

In [56]:
# Function DTorRF to check to see which Tree is better at default classification tree settings.
# for pipeline() we are using Standardscaler() because all columns are numeric and no values < 0.
def DTorRF(data, target, runs):

    X = np.array(df.drop(columns={target}))
    y = np.array(df[target]).reshape(-1,1)

    #X = (df.drop(columns={target})).values
    print('X Shape:',X.shape)
    #y = (df[target]).values
    print('Y Shape:',y.shape)


    X_train, X_test, y_train, y_test = train_test_split(X, y.ravel(), test_size=0.25, random_state=0)
    # create the pipeline
    pipeline = Pipeline([('scaler', StandardScaler())]) # using standardscaler()

    # fit the pipeline to the train data
    pipeline.fit(X_train)

    # transform the train and test data
    X_train_transformed = pipeline.transform(X_train)
    X_test_transformed = pipeline.transform(X_test)

    # create lists to compile the accuracies for each tree
    dt_accuracies = [] 
    rf_accuracies = []
    t=runs
    
    # perform the test for r trails: r being the number of trails wanted.
    for i in range(t):
        # Decision Tree
        dt = DecisionTreeClassifier()
        dt.fit(X_train_transformed, y_train.ravel())
        dt_pred = dt.predict(X_test_transformed)
        dt_acc = accuracy_score(y_test, dt_pred)
        dt_accuracies.append(dt_acc)

        # Random Forest
        rf = RandomForestClassifier()
        rf.fit(X_train_transformed, y_train.ravel())
        rf_pred = rf.predict(X_test_transformed)
        rf_acc = accuracy_score(y_test, rf_pred)
        rf_accuracies.append(rf_acc)

    # print the mean accuracy for each model
    # Calculating the mean of the all accuracies from trails.
    print(f"At default settings, Decision Tree Mean Accuracy after {t} runs: ", np.mean(dt_accuracies)) 
    print(f"At default settings, Random Forest Mean Accuracy after {t} runs: ", np.mean(rf_accuracies))

    # Deduce Clasisfication model based on highest mean score for each Tree used.
    if (np.mean(dt_accuracies))> (np.mean(rf_accuracies)):
        print()
        print('Looking at the results above (@ default settings), we will use Decision Tree Classification')
    else:
        print()
        print('Looking at the results above (@ default settings), we will use Randon Forest Classification.')
    return
    
DTorRF(df, 'target', 25)

X Shape: (250, 200)
Y Shape: (250, 1)
At default settings, Decision Tree Mean Accuracy after 25 runs:  0.52
At default settings, Random Forest Mean Accuracy after 25 runs:  0.6285714285714286

Looking at the results above (@ default settings), we will use Randon Forest Classification.


#### Do Modelling Stuff

Make a tree model (of some vareity) and make it fit well. Keep in mind the possibility of your tree overfitting, and think of steps you may need to combat that shoudl it occur. 

In [57]:
# redo the above, but with a pipeline
# Since all the columns are numeric and have NO null values, hence we only need to use the StandardScaler() or RobustScaler().fit
# Why Standardscaler?
# StandardScaler() follows Standard Normal Distribution (SND). Therefore, it makes mean = 0 and scales the data to unit variance. 
# MinMaxScaler() scales all the data features in the range [0, 1] or else in the range [-1, 1] if there are negative values in the dataset.
# RobustScaler() This Scaler removes the median and scales the data according to the quantile range (defaults to the IQR range)

estimator=RandomForestClassifier(n_jobs=-1)

scaler_RS=preprocessing.RobustScaler() 
scaler_SS=preprocessing.StandardScaler()
scaler_MM=preprocessing.MinMaxScaler()

#Create Pipeline Steps
pipeline_steps=[('pre', scaler_SS), ('rf', estimator)]
pipe=Pipeline(pipeline_steps)

# Split dataframe
X=np.array(df.drop(columns={'target'}))
y=np.array(df['target']).reshape(-1,1) # using np.array().reshape(-1,1) because some sklearn classification models require a 2D array.

#X = (df.drop(columns={'target'})).values
#y = (df['target']).values


#test, train split
#using ravel() because of value error in previous code.
# ravel() is used to create a contiguous flattened array.
X_train, X_test, y_train, y_test = train_test_split(X, y.ravel(), test_size=0.3, random_state=0)

# Hyperparameter Tuning
# n_estimators: The n_estimators parameter specifies the number of trees in the forest of the model. The default value for this parameter is 10, which means that 10 different decision trees will be constructed in the random forest.
# max_depth: The max_depth parameter specifies the maximum depth of each Decision Tree. The default value for max_depth is None, which means that each tree will expand until every leaf is pure. A pure leaf is one where all of the data on the leaf comes from the same class.
# min_samples_split: The min_samples_split parameter specifies the minimum number of samples required to split an internal leaf node. The default value for this parameter is 2, which means that an internal node must have at least two samples before it can be split to have a more specific classification.
# min_samples_leaf: The min_samples_leaf parameter specifies the minimum number of samples required to be at a leaf node. The default value for this parameter is 1, which means that every leaf must have at least 1 sample that it classifies.

rf_para = {'rf__min_samples_split':[2,3,4],
            'rf__max_depth':[3,4,5],
            'rf__n_estimators':[200, 210, 215, 220, 225],
            # 'rf__criterion':["Gini"],
            'rf__max_samples':[.0025, .05, .0525, .06]} # the number of samples to draw from X to train each base estimator

grid = GridSearchCV(pipe, param_grid=rf_para, cv=5, n_jobs=-1) # cv:Cross-Validation set at 5
grid.fit(X_train, y_train.ravel())
best=grid.best_estimator_ #  estimator which gave highest score (or smallest loss if specified) on the left out data.

In [ ]:
# Test 1(gini &/or Entropy): Took 26mins to run
#print(best.score(X_test, y_test))
#print(best)
#'rf__min_samples_split':[4,5,6,7,8],
            #'rf__max_depth':[3,4,5,6,7,8,9],
            #'rf__n_estimators':[80, 100, 150, 175, 200, 225],
            #'rf__criterion':["gini","entropy"], taking to long,
            #'rf__max_samples':[0.2,.3, .4, .5, .6, .7]

0.5866666666666667
Pipeline(steps=[('pre', StandardScaler()),
                ('rf',
                 RandomForestClassifier(max_depth=3, max_samples=0.2,
                                        min_samples_split=8, n_estimators=175,
                                        n_jobs=-1))])


In [ ]:
#test 2 (gini):
#print(best.score(X_test, y_test))
#print(best)

#{'rf__min_samples_split':[4,5,6], 
#'rf__max_depth':[2,3,4], 
#'rf__n_estimators':[180, 200, 210], 

#'rf__max_samples':[.4, .5, .6]} # the number of samples to draw from X to train each base estimator

0.6266666666666667
Pipeline(steps=[('pre', StandardScaler()),
                ('rf',
                 RandomForestClassifier(max_depth=3, max_samples=0.6,
                                        min_samples_split=4, n_estimators=225,
                                        n_jobs=-1))])


In [ ]:
#Test 3 (gini):
#print(best.score(X_test, y_test))
#print(best)

#'rf__min_samples_split':[3,4,5], 
#'rf__max_depth':[2,3,4], 
#'rf__n_estimators':[180, 200, 210, 215], 

#'rf__max_samples':[.4, .5, .6]} 

0.6
Pipeline(steps=[('pre', StandardScaler()),
                ('rf',
                 RandomForestClassifier(max_depth=3, max_samples=0.05,
                                        min_samples_split=5, n_estimators=215,
                                        n_jobs=-1))])


In [ ]:
#Test 4 (entropy)
#print(best.score(X_test, y_test))
#print(best)

#'rf__min_samples_split':[3,4,5],
# 'rf__max_depth':[2,3,4],
# 'rf__n_estimators':[180, 200, 210, 215, 220],
# 'rf__criterion':["entropy"],
# 'rf__max_samples':[.01, .02, .03, .05]}

0.5066666666666667
Pipeline(steps=[('pre', StandardScaler()),
                ('rf',
                 RandomForestClassifier(criterion='entropy', max_depth=3,
                                        max_samples=0.05, min_samples_split=5,
                                        n_estimators=210, n_jobs=-1))])


In [58]:
#Test 5 (gini)
#print(best.score(X_test, y_test))
#print(best)

#'rf__min_samples_split':[2,3,4],
# 'rf__max_depth':[3,4,5],
# 'rf__n_estimators':[200, 210, 215, 220, 225],
# 'rf__criterion':["Gini"],
# 'rf__max_samples':[.0025, .05, .0525, .06]}

0.5333333333333333
Pipeline(steps=[('pre', StandardScaler()),
                ('rf',
                 RandomForestClassifier(max_depth=3, max_samples=0.06,
                                        min_samples_split=4, n_estimators=215,
                                        n_jobs=-1))])


### Finishing

At the conclusion, please name your best model "best". If you look down below in the testing stuff, it should be usable to score as "best". 

You should be able to call it like this and it should work (with whatever data names you have)

In [59]:
print(best.score(X_test, y_test)) #using Test 5 results
print(best)

0.5333333333333333
Pipeline(steps=[('pre', StandardScaler()),
                ('rf',
                 RandomForestClassifier(max_depth=3, max_samples=0.06,
                                        min_samples_split=4, n_estimators=215,
                                        n_jobs=-1))])


### Testing

Please leave the stuff below as-is in your file. 

This will take your best model and score it with the test data. If you want to test to make sure that yours works, make a copy of the data file and rename it testing.csv, then make sure this runs ok. I will do the same, but the contents of my test file will be different. 

In [60]:
#Load Test Data
test_df = pd.read_csv("azimTEST.csv")
#test_df = test_df.drop(columns={"id"}) # Please uncheck this when using your test file.
#Create tests and score
test_y = np.array(test_df["target"]).reshape(-1,1)
test_X = np.array(test_df.drop(columns={"target"}))

preds = best.predict(test_X)

roc_score = roc_auc_score(test_y, preds)
acc_score = accuracy_score(test_y, preds)

print(roc_score)
print(acc_score)
print(name, np.mean([roc_score, acc_score]))


0.730393825899444
0.7393617021276596
Azim's 0.7348777640135518


### What Accuracy Changes Were Used

Please list here what you did to try to increase accuracy and/or limit overfitting:

<ul>
<li>Increase Accuracy: Increase the Number of Decision Trees within a Forest (By increasing the n_estimators)
<li>Limit Overfitting: Reducing Tree depth (By decreasing max_depth)
<li>Limit Overfitting: Reducing the number of Variables samples at each split:
<li>Limit Overfitting: Using more data: Unfortunately we dont have have more training data.
</ul>